In [1]:
from training_tools.train_models import train_models_by_years
from training_tools.evaluate_models import evaluate_models_by_years

# **Train `Word2vec` Models**

In [ ]:
train_models_by_years(
    data_dir=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng/'
        '5gram_files/6corpus/yearly_files'
    ),
    years=(2019, 2019),
    weight_by='none',
    log_base=10,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4
)

Training Models:   0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
evaluate_models_by_years(
    data_dir=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng/'
        '5gram_files/6corpus/yearly_files'
    ),
    years=(1700, 1700)
)

{'w2v_1700_freq_100_5_1.model': {'similarity_score': 0.3082932782849958,
  'analogy_score': 0.04092261904761905}}